In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# %matplotlib inline
#from config import path
#add
import gc
path = '//Users/xuelili/eguan data/Demo/'
packtime = pd.read_table(path+'deviceid_package_start_close.tsv',
                         names=['device_id', 'app', 'start', 'close'], low_memory=True)
# packtime.head()
packtime['peroid'] = (packtime['close'] - packtime['start'])/1000
packtime['start'] = pd.to_datetime(packtime['start'], unit='ms')
#packtime['closetime'] = pd.to_datetime(packtime['close'], unit='ms')
del packtime['close']
gc.collect()

#packtime['day'] = packtime['start'].dt.day
#packtime['month'] = packtime['start'].dt.month
packtime['hour'] = packtime['start'].dt.hour
packtime['date'] = packtime['start'].dt.date
packtime['dayofweek'] = packtime['start'].dt.dayofweek
#packtime['hour'] = pd.cut(packtime['hour'], bins=4).cat.codes

#平均每天使用设备时间
dtime = packtime.groupby(['device_id', 'date'])['peroid'].agg('sum')
#不同时间段占比
qtime = packtime.groupby(['device_id', 'hour'])['peroid'].agg('sum')
wtime = packtime.groupby(['device_id', 'dayofweek'])['peroid'].agg('sum')
atime = packtime.groupby(['device_id', 'app'])['peroid'].agg('sum')


dapp = packtime[['device_id', 'date', 'app']].drop_duplicates().groupby(
    ['device_id', 'date'])['app'].agg(' '.join)
dapp = dapp.reset_index()
dapp['app_len'] = dapp['app'].apply(lambda x: x.split(' ')).apply(len)
dapp_stat = dapp.groupby('device_id')['app_len'].agg(
    {'std': 'std', 'mean': 'mean', 'max': 'max'})
dapp_stat = dapp_stat.reset_index()
dapp_stat.columns = ['device_id', 'app_len_std', 'app_len_mean', 'app_len_max']
# dapp_stat.head()

dtime = dtime.reset_index()
dtime_stat = dtime.groupby(['device_id'])['peroid'].agg(
    {'sum': 'sum', 'mean': 'mean', 'std': 'std', 'max': 'max'}).reset_index()
dtime_stat.columns = ['device_id', 'date_sum',
                      'date_mean', 'date_std', 'date_max']
# dtime_stat.head()

qtime = qtime.reset_index()
ftime = qtime.pivot(index='device_id', columns='hour',
                    values='peroid').fillna(0)
ftime.columns = ['h%s' % i for i in range(24)]
ftime.reset_index(inplace=True)
# ftime.head()

wtime = wtime.reset_index()
weektime = wtime.pivot(
    index='device_id', columns='dayofweek', values='peroid').fillna(0)
weektime.columns = ['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6']
weektime.reset_index(inplace=True)
# weektime.head()

atime = atime.reset_index()
app = atime.groupby(['device_id'])['peroid'].idxmax()

#dapp_stat.shape, dtime_stat.shape, ftime.shape, weektime.shape, app.shape

user = pd.merge(dapp_stat, dtime_stat, on='device_id', how='left')
user = pd.merge(user, ftime, on='device_id', how='left')
user = pd.merge(user, weektime, on='device_id', how='left')
user = pd.merge(user, atime.iloc[app], on='device_id', how='left')

app_cat = pd.read_table(path+'package_label.tsv',
                        names=['app', 'category', 'app_name'])

cat_enc = pd.DataFrame(app_cat['category'].value_counts())
cat_enc['idx'] = range(45)

app_cat['cat_enc'] = app_cat['category'].map(cat_enc['idx'])
app_cat.set_index(['app'], inplace=True)

atime['app_cat_enc'] = atime['app'].map(app_cat['cat_enc']).fillna(45)

cat_num = atime.groupby(['device_id', 'app_cat_enc'])[
    'app'].agg('count').reset_index()
cat_time = atime.groupby(['device_id', 'app_cat_enc'])[
    'peroid'].agg('sum').reset_index()

app_cat_num = cat_num.pivot(
    index='device_id', columns='app_cat_enc', values='app').fillna(0)
app_cat_num.columns = ['cat%s' % i for i in range(46)]
app_cat_time = cat_time.pivot(
    index='device_id', columns='app_cat_enc', values='peroid').fillna(0)
app_cat_time.columns = ['time%s' % i for i in range(46)]

user = pd.merge(user, app_cat_num, on='device_id', how='left')
user = pd.merge(user, app_cat_time, on='device_id', how='left')
user.to_csv('/Users/xuelili/eguan data/feature1/user_behavior.csv', index=False)




/Users/xuelili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
/Users/xuelili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [7]:
# coding: utf-8

# In[ ]:


import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from datetime import datetime,timedelta  
import matplotlib.pyplot as plt
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# %matplotlib inline

#add
import gc
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import hstack, vstack
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from skopt.space import Integer, Categorical, Real, Log10
from skopt.utils import use_named_args
from skopt import gp_minimize
from gensim.models import Word2Vec, FastText
import gensim 
import re


# In[ ]:


test = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_test.tsv', sep='\t', names=['device_id'])
train = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_train.tsv', sep='\t', names=['device_id', 'sex', 'age'])
brand = pd.read_table('/Users/xuelili/eguan data/Demo/deviceid_brand.tsv', names=['device_id', 'vendor', 'version'])
packtime = pd.read_table('/Users/xuelili/eguan data/Demo/deviceid_package_start_close.tsv', 
                         names=['device_id', 'app', 'start', 'close'])
packages = pd.read_csv('/Users/xuelili/eguan data/Demo/deviceid_packages.tsv', sep='\t', names=['device_id', 'apps'])


# In[ ]:


packtime['period'] = (packtime['close'] - packtime['start'])/1000
packtime['start'] = pd.to_datetime(packtime['start'], unit='ms')
app_use_time = packtime.groupby(['app'])['period'].agg('sum').reset_index()
# 试试看200
app_use_top100 = app_use_time.sort_values(by='period', ascending=False)[:100]['app']
device_app_use_time = packtime.groupby(['device_id', 'app'])['period'].agg('sum').reset_index()
use_time_top100_statis = device_app_use_time.set_index('app').loc[list(app_use_top100)].reset_index()
top100_statis = use_time_top100_statis.pivot(index='device_id', columns='app', values='period').reset_index()


# In[ ]:


top100_statis = top100_statis.fillna(0)


# In[ ]:


# 手机品牌预处理
brand['vendor'] = brand['vendor'].astype(str).apply(lambda x : x.split(' ')[0].upper())
brand['ph_ver'] = brand['vendor'] + '_' + brand['version']

ph_ver = brand['ph_ver'].value_counts()
ph_ver_cnt = pd.DataFrame(ph_ver).reset_index()
ph_ver_cnt.columns = ['ph_ver', 'ph_ver_cnt']

brand = pd.merge(left=brand, right=ph_ver_cnt,on='ph_ver')


# In[ ]:


# 针对长尾分布做的一点处理
mask = (brand.ph_ver_cnt < 100)
brand.loc[mask, 'ph_ver'] = 'other' 

train = pd.merge(brand[['device_id', 'ph_ver']], train, on='device_id', how='right')
test = pd.merge(brand[['device_id', 'ph_ver']], test, on='device_id', how='right')
train['ph_ver'] = train['ph_ver'].astype(str)
test['ph_ver'] = test['ph_ver'].astype(str)

# 将 ph_ver 进行 label encoder
ph_ver_le = preprocessing.LabelEncoder()
train['ph_ver'] = ph_ver_le.fit_transform(train['ph_ver'])
test['ph_ver'] = ph_ver_le.transform(test['ph_ver'])
train['label'] = train['sex'].astype(str) + '-' + train['age'].astype(str)
label_le = preprocessing.LabelEncoder()
train['label'] = label_le.fit_transform(train['label'])


# In[ ]:


test['sex'] = -1
test['age'] = -1
test['label'] = -1
data = pd.concat([train, test], ignore_index=True)
data.shape


# In[ ]:


ph_ver_dummy = pd.get_dummies(data['ph_ver'])
ph_ver_dummy.columns = ['ph_ver_' + str(i) for i in range(ph_ver_dummy.shape[1])]


# In[ ]:


data = pd.concat([data, ph_ver_dummy], axis=1)


# In[ ]:


del data['ph_ver']


# In[ ]:


train = data[data.sex != -1]
test = data[data.sex == -1]
train.shape, test.shape


# In[ ]:


# 每个app的总使用次数统计
app_num = packtime['app'].value_counts().reset_index()
app_num.columns = ['app', 'app_num']
packtime = pd.merge(left=packtime, right=app_num, on='app')
# 同样的，针对长尾分布做些处理（尝试过不做处理，或换其他阈值，这个100的阈值最高）
packtime.loc[packtime.app_num < 100, 'app'] = 'other'


# In[ ]:


# 统计每台设备的app数量
df_app = packtime[['device_id', 'app']]
apps = df_app.drop_duplicates().groupby(['device_id'])['app'].apply(' '.join).reset_index()
apps['app_length'] = apps['app'].apply(lambda x:len(x.split(' ')))

train = pd.merge(train, apps, on='device_id', how='left')
test = pd.merge(test, apps, on='device_id', how='left')


# In[ ]:


# 获取每台设备所安装的apps的tfidf
tfidf = CountVectorizer(lowercase=False, min_df=3, stop_words=top100_statis.columns.tolist()[1:7])
apps['app'] = tfidf.fit_transform(apps['app'])

X_tr_app = tfidf.transform(list(train['app']))
X_ts_app = tfidf.transform(list(test['app']))


# In[ ]:


'''
svd = TruncatedSVD(n_components=100, random_state=42)
X = vstack([X_tr_app, X_ts_app])
svd.fit(X)
X_tr_app = svd.fit_transform(X_tr_app)
X_ts_app = svd.fit_transform(X_ts_app)
X_tr_app = pd.DataFrame(X_tr_app)
X_ts_app = pd.DataFrame(X_ts_app)
X_tr_app.columns = ['app_' + str(i) for i in range(0, 100)]
X_ts_app.columns = ['app_' + str(i) for i in range(0, 100)]
'''


# ### 利用word2vec得到每台设备所安装app的embedding表示

# In[ ]:


packages['apps'] = packages['apps'].apply(lambda x:x.split(','))
packages['app_length'] = packages['apps'].apply(lambda x:len(x))


# In[ ]:


embed_size = 128
fastmodel = Word2Vec(list(packages['apps']), size=embed_size, window=4, min_count=3, negative=2,
                 sg=1, sample=0.002, hs=1, workers=4)  

embedding_fast = pd.DataFrame([fastmodel[word] for word in (fastmodel.wv.vocab)])
embedding_fast['app'] = list(fastmodel.wv.vocab)
embedding_fast.columns= ["fdim_%s" % str(i) for i in range(embed_size)]+["app"]
embedding_fast.head()


# In[ ]:


id_list = []
for i in range(packages.shape[0]):
    id_list += [list(packages['device_id'])[i]]*packages['app_length'].iloc[i]


app_list = [word for item in packages['apps'] for word in item]

app_vect = pd.DataFrame({'device_id':id_list})        
app_vect['app'] = app_list


# In[ ]:


app_vect = app_vect.merge(embedding_fast, on='app', how='left')
app_vect = app_vect.drop('app', axis=1)

seqfeature = app_vect.groupby(['device_id']).agg('mean')
seqfeature.reset_index(inplace=True)


# In[ ]:


seqfeature.head()


# ### 用户一周七天玩手机的时长情况

# In[ ]:


# packtime['period'] = (packtime['close'] - packtime['start'])/1000
# packtime['start'] = pd.to_datetime(packtime['start'], unit='ms')
packtime['dayofweek'] = packtime['start'].dt.dayofweek
packtime['hour'] = packtime['start'].dt.hour
# packtime = packtime[(packtime['start'] < '2017-03-31 23:59:59') & (packtime['start'] > '2017-03-01 00:00:00')]


# In[ ]:


app_use_time = packtime.groupby(['device_id', 'dayofweek'])['period'].agg('sum').reset_index()
week_app_use = app_use_time.pivot_table(values='period', columns='dayofweek', index='device_id').reset_index()
week_app_use = week_app_use.fillna(0)
week_app_use.columns = ['device_id'] + ['week_day_' + str(i) for i in range(0, 7)]

week_app_use['week_max'] = week_app_use.max(axis=1)
week_app_use['week_min'] = week_app_use.min(axis=1)
week_app_use['week_sum'] = week_app_use.sum(axis=1)
week_app_use['week_std'] = week_app_use.std(axis=1)

'''
for i in range(0, 7):
    week_app_use['week_day_' + str(i)] = week_app_use['week_day_' + str(i)] / week_app_use['week_sum']
'''


# In[ ]:


'''
app_use_time = packtime.groupby(['device_id', 'hour'])['period'].agg('sum').reset_index()
hour_app_use = app_use_time.pivot_table(values='period', columns='hour', index='device_id').reset_index()
hour_app_use = hour_app_use.fillna(0)
hour_app_use.columns = ['device_id'] + ['hour_' + str(i) for i in range(0, 24)]

# hour_app_use['hour_max'] = hour_app_use.max(axis=1)
# hour_app_use['hour_min'] = hour_app_use.min(axis=1)
# hour_app_use['hour_sum'] = hour_app_use.sum(axis=1)
# hour_app_use['hour_std'] = hour_app_use.std(axis=1)

# for i in range(0, 24):
#     hour_app_use['hour_' + str(i)] = hour_app_use['hour_' + str(i)] / hour_app_use['hour_sum']
'''


# ### 将各个特征整合到一块

# In[ ]:


train.columns[4:]


# In[ ]:


user_behavior = pd.read_csv('/Users/xuelili/eguan data/feature1/user_behavior.csv')
user_behavior['app_len_max'] = user_behavior['app_len_max'].astype(np.float64)
del user_behavior['app']
train = pd.merge(train, user_behavior, on='device_id', how='left')
test = pd.merge(test, user_behavior, on='device_id', how='left')


# In[ ]:


train = pd.merge(train, seqfeature, on='device_id', how='left')
test = pd.merge(test, seqfeature, on='device_id', how='left')


# In[ ]:


train = pd.merge(train, week_app_use, on='device_id', how='left')
test = pd.merge(test, week_app_use, on='device_id', how='left')


# In[ ]:


'''
app_top50_list = list(packtime.groupby(by='app')['period'].sum().sort_values(ascending=False)[:50].index)

for app in app_top50_list:
    app_cnt = packtime[packtime['app'] == app]
    start_num_app = app_cnt.groupby(by='device_id')['start'].count().reset_index()
    start_num_app.columns = ['device_id', 'start_num_app_' + app[0:4]]
    train = train.merge(start_num_app, on='device_id', how='left')
    test = test.merge(start_num_app, on='device_id', how='left')
    print(app + ' done')   
'''


# In[ ]:


'''
# all_top50 : 使用总时长最高的50款app，每个人的使用时间统计
all_top50 = pd.read_csv('../input/yg-feature/all_top50_statis.csv')
train = pd.merge(train, all_top50, on='device_id', how='left')
test = pd.merge(test, all_top50, on='device_id', how='left')
'''


# In[ ]:


top100_statis.columns = ['device_id'] + ['top100_statis_' + str(i) for i in range(0, 100)]
train = pd.merge(train, top100_statis, on='device_id', how='left')
test = pd.merge(test, top100_statis, on='device_id', how='left')


# In[ ]:


train.to_csv('/Users/xuelili/eguan data/feature1/train_feature.csv', index=None)
test.to_csv('/Users/xuelili/eguan data/feature1/test_feature.csv', index=None)


# In[ ]:


feats = train.columns[4:]
feats


# In[ ]:


feats = feats.delete(153)
feats[153]


# In[ ]:


'''
train = pd.merge(train, hour_app_use, on='device_id', how='left')
test = pd.merge(test, hour_app_use, on='device_id', how='left')
'''


# In[ ]:


X_train = hstack([X_tr_app, train[feats].astype(float)])
X_test = hstack([X_ts_app, test[feats].astype(float)])

X_train = X_train.tocsr().astype('float')
X_test = X_test.tocsr().astype('float')


# ### 开始训练模型

# In[ ]:


Y = train['sex'] - 1
kfold = StratifiedKFold(n_splits=10, random_state=10, shuffle=True)
oof_preds1 = np.zeros((X_train.shape[0], ))
sub1 = np.zeros((X_test.shape[0], ))
for i, (train_index, test_index) in enumerate(kfold.split(X_train, Y)): 
    X_tr, X_vl, y_tr, y_vl = X_train[train_index], X_train[test_index],                                 Y[train_index], Y[test_index]
    dtrain = lgb.Dataset(X_tr, label=y_tr)
    dvalid = lgb.Dataset(X_vl, y_vl, reference=dtrain)
    params = {
        'boosting_type': 'gbdt',
        'max_depth':6,
        'objective':'binary',
        'num_leaves':31,
        'subsample': 0.85,
        'colsample_bytree': 0.2,
        'lambda_l1':0.00007995302080034896,
        'lambda_l2':0.0003648648811380991,
        'subsample_freq':12,
        'learning_rate': 0.012,
        'min_child_weight':5.5
    }

    model = lgb.train(params,
                        dtrain,
                        num_boost_round=4000,
                        valid_sets=dvalid,
                        early_stopping_rounds=100,
                        verbose_eval=100)

    oof_preds1[test_index] = model.predict(X_vl, num_iteration=model.best_iteration)
    sub1 += model.predict(X_test, num_iteration=model.best_iteration)/kfold.n_splits


# In[ ]:


Y = train['age']
kfold = StratifiedKFold(n_splits=10, random_state=10, shuffle=True)
oof_preds2 = np.zeros((X_train.shape[0], 11))
sub2 = np.zeros((X_test.shape[0], 11))
for i, (train_index, test_index) in enumerate(kfold.split(X_train, Y)):
    X_tr, X_vl, y_tr, y_vl = X_train[train_index], X_train[test_index],                                 Y[train_index], Y[test_index]
    dtrain = lgb.Dataset(X_tr, label=y_tr)
    dvalid = lgb.Dataset(X_vl, y_vl, reference=dtrain)
    params = {
        'boosting_type': 'gbdt',
        'max_depth':6,
        'metric': {'multi_logloss'},
        'num_class':11,
        'objective':'multiclass',
        'num_leaves':31,
        'subsample': 0.9,
        'colsample_bytree': 0.2,
        'lambda_l1':0.0001,
        'lambda_l2':0.00111,
        'subsample_freq':10,
        'learning_rate': 0.012,
        'min_child_weight':10
    }

    model = lgb.train(params,
                        dtrain,
                        num_boost_round=4000,
                        valid_sets=dvalid,
                        early_stopping_rounds=100,
                        verbose_eval=100)

    oof_preds2[test_index] = model.predict(X_vl, num_iteration=model.best_iteration)
    sub2 += model.predict(X_test, num_iteration=model.best_iteration)/kfold.n_splits


# In[ ]:


oof_preds1 = pd.DataFrame(oof_preds1, columns=['sex2'])

oof_preds1['sex1'] = 1-oof_preds1['sex2']
oof_preds2 = pd.DataFrame(oof_preds2, columns=['age%s'%i for i in range(11)])
oof_preds = train[['device_id']]
oof_preds.columns = ['DeviceID']

for i in ['sex1', 'sex2']:
    for j in ['age%s'%i for i in range(11)]:
        oof_preds[i+'_'+j] = oof_preds1[i] * oof_preds2[j]
oof_preds.columns = ['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', 
         '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', 
         '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']

oof_preds.to_csv('/Users/xuelili/eguan data/feature1/train.csv', index=False)


# In[ ]:


sub1 = pd.DataFrame(sub1, columns=['sex2'])

sub1['sex1'] = 1-sub1['sex2']
sub2 = pd.DataFrame(sub2, columns=['age%s'%i for i in range(11)])
sub = test[['device_id']]
sub.columns = ['DeviceID']

for i in ['sex1', 'sex2']:
    for j in ['age%s'%i for i in range(11)]:
        sub[i+'_'+j] = sub1[i] * sub2[j]
sub.columns = ['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', 
         '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', 
         '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']

sub.to_csv('/Users/xuelili/eguan data/feature1/lgb_l_v54.csv', index=False)


# In[ ]:


'''
Y = train['label']
#best params: [31, 11, 0.015955854914003094, 0.12122664084283229, 0.7645440142264772, 24, 1048, 0.00552258737237652, 0.005810068328090833, 7]
kfold = StratifiedKFold(n_splits=5, random_state=10, shuffle=True)
sub = np.zeros((X_test.shape[0], 22))
for i, (train_index, test_index) in enumerate(kfold.split(X_train, Y)):
    X_tr, X_vl, y_tr, y_vl = X_train[train_index], X_train[test_index], Y[train_index], Y[test_index]
    dtrain = lgb.Dataset(X_tr, label=y_tr)
    dvalid = lgb.Dataset(X_vl, y_vl, reference=dtrain)
    params = {
        'boosting_type': 'gbdt',
        'max_depth':7,
        'objective':'multiclass',
        'metric': {'multi_logloss'},
        'num_class':22,
        'num_leaves':20,
        'subsample': 0.86,
        'colsample_bytree': 0.8,
        #'lambda_l1':0.00007995302080034896,
        'lambda_l2':0.005,
        'subsample_freq':11,
        'learning_rate': 0.01,
        'min_child_weight':5.5,

    }
    
    model = lgb.train(params,
                        dtrain,
                        num_boost_round=6000,
                        valid_sets=dvalid,
                        early_stopping_rounds=20,
                        verbose_eval=100)


    sub += model.predict(X_test, num_iteration=model.best_iteration)/kfold.n_splits
'''


# In[ ]:


'''
sub = pd.DataFrame(sub)
cols = [x for x in range(0, 22)]
cols = label_le.inverse_transform(cols)

sub.columns = cols
sub['DeviceID'] = test['device_id'].values

sub = sub[['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', 
         '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', 
         '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']]

sub.to_csv('30.csv', index=False)
'''




/Users/xuelili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:205: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.617899
[200]	valid_0's binary_logloss: 0.609378
[300]	valid_0's binary_logloss: 0.605883
[400]	valid_0's binary_logloss: 0.60386
[500]	valid_0's binary_logloss: 0.602927
[600]	valid_0's binary_logloss: 0.602526
[700]	valid_0's binary_logloss: 0.602121
[800]	valid_0's binary_logloss: 0.601841
[900]	valid_0's binary_logloss: 0.601678
[1000]	valid_0's binary_logloss: 0.601643
Early stopping, best iteration is:
[939]	valid_0's binary_logloss: 0.601561
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.619835
[200]	valid_0's binary_logloss: 0.611707
[300]	valid_0's binary_logloss: 0.608511
[400]	valid_0's binary_logloss: 0.60714
[500]	valid_0's binary_logloss: 0.606246
[600]	valid_0's binary_logloss: 0.605544
[700]	valid_0's binary_logloss: 0.605278
[800]	valid_0's binary_logloss: 0.605058
[900]	valid_0's binary_logloss: 0.604895
[1000]	valid_0's bin

[1100]	valid_0's multi_logloss: 1.98209
[1200]	valid_0's multi_logloss: 1.98153
[1300]	valid_0's multi_logloss: 1.98122
Early stopping, best iteration is:
[1299]	valid_0's multi_logloss: 1.98121
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 2.16662
[200]	valid_0's multi_logloss: 2.08366
[300]	valid_0's multi_logloss: 2.04654
[400]	valid_0's multi_logloss: 2.02891
[500]	valid_0's multi_logloss: 2.01927
[600]	valid_0's multi_logloss: 2.01389
[700]	valid_0's multi_logloss: 2.01065
[800]	valid_0's multi_logloss: 2.00864
[900]	valid_0's multi_logloss: 2.00713
[1000]	valid_0's multi_logloss: 2.00589
[1100]	valid_0's multi_logloss: 2.00529
[1200]	valid_0's multi_logloss: 2.00509
Early stopping, best iteration is:
[1198]	valid_0's multi_logloss: 2.00505
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 2.15367
[200]	valid_0's multi_logloss: 2.06469
[300]	valid_0's multi_logloss: 2.02502
[400]	valid_0's 

/Users/xuelili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/xuelili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:506: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


"\nsub = pd.DataFrame(sub)\ncols = [x for x in range(0, 22)]\ncols = label_le.inverse_transform(cols)\n\nsub.columns = cols\nsub['DeviceID'] = test['device_id'].values\n\nsub = sub[['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', \n         '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', \n         '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']]\n\nsub.to_csv('30.csv', index=False)\n"

In [12]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# %matplotlib inline

#add
from category_encoders import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
#from config import path
path = '/Users/xuelili/eguan data/Demo/'
packages = pd.read_csv(path+'deviceid_packages.tsv',
                       sep='\t', names=['device_id', 'apps'])
test = pd.read_csv(path+'deviceid_test.tsv',
                   sep='\t', names=['device_id'])
train = pd.read_csv(path+'deviceid_train.tsv',
                    sep='\t', names=['device_id', 'sex', 'age'])

brand = pd.read_table(path+'deviceid_brand.tsv',
                      names=['device_id', 'vendor', 'version'])
behave = pd.read_csv('/Users/xuelili/eguan data/feature1/user_behavior.csv')

brand['phone_version'] = brand['vendor'] + ' ' + brand['version']
train = pd.merge(brand[['device_id', 'phone_version']],
                 train, on='device_id', how='right')
test = pd.merge(brand[['device_id', 'phone_version']],
                test, on='device_id', how='right')

train = pd.merge(train, behave, on='device_id', how='left')
test = pd.merge(test, behave, on='device_id', how='left')

packages['app_lenghth'] = packages['apps'].apply(
    lambda x: x.split(',')).apply(lambda x: len(x))
packages['app_list'] = packages['apps'].apply(lambda x: x.split(','))
train = pd.merge(train, packages, on='device_id', how='left')
test = pd.merge(test, packages, on='device_id', how='left')

embed_size = 128
fastmodel = Word2Vec(list(packages['app_list']), size=embed_size, window=4, min_count=3, negative=2,
                     sg=1, sample=0.002, hs=1, workers=4)

embedding_fast = pd.DataFrame([fastmodel[word]
                               for word in (fastmodel.wv.vocab)])
embedding_fast['app'] = list(fastmodel.wv.vocab)
embedding_fast.columns = ["fdim_%s" %
                          str(i) for i in range(embed_size)]+["app"]


tokenizer = Tokenizer(lower=False, char_level=False, split=',')

tokenizer.fit_on_texts(list(packages['apps']))

X_seq = tokenizer.texts_to_sequences(train['apps'])
X_test_seq = tokenizer.texts_to_sequences(test['apps'])

maxlen = 50
X = pad_sequences(X_seq, maxlen=maxlen, value=0)
X_test = pad_sequences(X_test_seq, maxlen=maxlen, value=0)
Y_sex = train['sex']-1

max_feaures = 35001
embedding_matrix = np.zeros((max_feaures, embed_size))
for word in tokenizer.word_index:
    if word not in fastmodel.wv.vocab:
        continue
    embedding_matrix[tokenizer.word_index[word]] = fastmodel[word]


X_h = train[['h%s' % i for i in range(24)]].values
X_h_test = test[['h%s' % i for i in range(24)]].values


class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def model_conv1D(embedding_matrix):

    K.clear_session()
    # The embedding layer containing the word vectors
    emb_layer = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=maxlen,
        trainable=False
    )
    lstm_layer = Bidirectional(
        GRU(128, recurrent_dropout=0.15, dropout=0.15, return_sequences=True))

    # 1D convolutions that can iterate over the word vectors
    conv1 = Conv1D(filters=128, kernel_size=1,
                   padding='same', activation='relu',)
    conv2 = Conv1D(filters=64, kernel_size=2,
                   padding='same', activation='relu', )
    conv3 = Conv1D(filters=64, kernel_size=3,
                   padding='same', activation='relu',)
    conv5 = Conv1D(filters=32, kernel_size=5,
                   padding='same', activation='relu',)

    # Define inputs
    seq = Input(shape=(maxlen,))

    # Run inputs through embedding
    emb = emb_layer(seq)

    lstm = lstm_layer(emb)
    # Run through CONV + GAP layers
    conv1a = conv1(lstm)
    gap1a = GlobalAveragePooling1D()(conv1a)
    gmp1a = GlobalMaxPool1D()(conv1a)

    conv2a = conv2(lstm)
    gap2a = GlobalAveragePooling1D()(conv2a)
    gmp2a = GlobalMaxPool1D()(conv2a)

    conv3a = conv3(lstm)
    gap3a = GlobalAveragePooling1D()(conv3a)
    gmp3a = GlobalMaxPooling1D()(conv3a)

    conv5a = conv5(lstm)
    gap5a = GlobalAveragePooling1D()(conv5a)
    gmp5a = GlobalMaxPooling1D()(conv5a)

    hin = Input(shape=(24, ))
    htime = Dense(6, activation='relu')(hin)
    merge1 = concatenate([gmp1a, gmp1a, gmp1a, gmp1a, htime])

    # The MLP that determines the outcome
    x = Dropout(0.3)(merge1)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    pred = Dense(1, activation='sigmoid')(x)

    # model = Model(inputs=[seq1, seq2, magic_input, distance_input], outputs=pred)
    model = Model(inputs=[seq, hin], outputs=pred)
    model.compile(loss='binary_crossentropy',
                  optimizer=AdamW(weight_decay=0.08,))

    return model


kfold = StratifiedKFold(n_splits=5, random_state=20, shuffle=True)
sub1 = np.zeros((X_test.shape[0], ))
oof_pref1 = np.zeros((X.shape[0], 1))
score = []
count = 0
for i, (train_index, test_index) in enumerate(kfold.split(X, Y_sex)):
    print("FOLD | ", count+1)
    filepath = "sex_weights_best_%d.h5" % count
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.8, patience=2, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=6, verbose=1, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]

    model_sex = model_conv1D(embedding_matrix)
    X_tr, X_vl, X_tr2, X_vl2, y_tr, y_vl = X[train_index], X[test_index], X_h[
        train_index], X_h[test_index], Y_sex[train_index], Y_sex[test_index]
    hist = model_sex.fit([X_tr, X_tr2], y_tr, batch_size=256, epochs=50, validation_data=([X_vl, X_vl2], y_vl),
                         callbacks=callbacks, verbose=1, shuffle=True)
    model_sex.load_weights(filepath)
    sub1 += np.squeeze(model_sex.predict([X_test, X_h_test]))/kfold.n_splits
    oof_pref1[test_index] = model_sex.predict([X_vl, X_vl2])
    score.append(np.min(hist.history['val_loss']))
    count += 1
print('log loss:', np.mean(score))


oof_pref1 = pd.DataFrame(oof_pref1, columns=['sex2'])
sub1 = pd.DataFrame(sub1, columns=['sex2'])
res1 = pd.concat([oof_pref1, sub1])
res1['sex1'] = 1-res1['sex2']
# res1.to_csv("res1.csv", index=False)


def model_age_conv(embedding_matrix):

    # The embedding layer containing the word vectors
    K.clear_session()
    emb_layer = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=maxlen,
        trainable=False
    )
    lstm_layer = Bidirectional(
        GRU(128, recurrent_dropout=0.15, dropout=0.15, return_sequences=True))

    # 1D convolutions that can iterate over the word vectors
    conv1 = Conv1D(filters=128, kernel_size=1,
                   padding='same', activation='relu',)
    conv2 = Conv1D(filters=64, kernel_size=2,
                   padding='same', activation='relu', )
    conv3 = Conv1D(filters=64, kernel_size=3,
                   padding='same', activation='relu',)
    conv5 = Conv1D(filters=32, kernel_size=5,
                   padding='same', activation='relu',)

    # Define inputs
    seq = Input(shape=(maxlen,))

    # Run inputs through embedding
    emb = emb_layer(seq)

    lstm = lstm_layer(emb)
    # Run through CONV + GAP layers
    conv1a = conv1(lstm)
    gap1a = GlobalAveragePooling1D()(conv1a)
    gmp1a = GlobalMaxPool1D()(conv1a)

    conv2a = conv2(lstm)
    gap2a = GlobalAveragePooling1D()(conv2a)
    gmp2a = GlobalMaxPool1D()(conv2a)

    conv3a = conv3(lstm)
    gap3a = GlobalAveragePooling1D()(conv3a)
    gmp3a = GlobalMaxPooling1D()(conv3a)

    conv5a = conv5(lstm)
    gap5a = GlobalAveragePooling1D()(conv5a)
    gmp5a = GlobalMaxPooling1D()(conv5a)

    merge1 = concatenate([gap1a, gap2a, gap3a, gap5a])

    # The MLP that determines the outcome
    x = Dropout(0.3)(merge1)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    x = Dense(200, activation='relu',)(x)
    x = Dropout(0.22)(x)
    x = BatchNormalization()(x)
    pred = Dense(11, activation='softmax')(x)

    model = Model(inputs=seq, outputs=pred)
    model.compile(loss='categorical_crossentropy',
                  optimizer=AdamW(weight_decay=0.08,))

    return model


Y_age = to_categorical(train['age'])

sub2 = np.zeros((X_test.shape[0], 11))
oof_pref2 = np.zeros((X.shape[0], 11))
score = []
count = 0
for i, (train_index, test_index) in enumerate(kfold.split(X, train['age'])):

    print("FOLD | ", count+1)

    filepath2 = "age_weights_best_%d.h5" % count
    checkpoint2 = ModelCheckpoint(
        filepath2, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr2 = ReduceLROnPlateau(
        monitor='val_loss', factor=0.8, patience=2, min_lr=0.0001, verbose=1)
    earlystopping2 = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=8, verbose=1, mode='auto')
    callbacks2 = [checkpoint2, reduce_lr2, earlystopping2]

    X_tr, X_vl, y_tr, y_vl = X[train_index], X[test_index], Y_age[train_index], Y_age[test_index]

    model_age = model_age_conv(embedding_matrix)
    hist = model_age.fit(X_tr, y_tr, batch_size=256, epochs=50, validation_data=(X_vl, y_vl),
                         callbacks=callbacks2, verbose=2, shuffle=True)

    model_age.load_weights(filepath2)
    oof_pref2[test_index] = model_age.predict(X_vl)
    sub2 += model_age.predict(X_test)/kfold.n_splits
    score.append(np.min(hist.history['val_loss']))
    count += 1
print('log loss:', np.mean(score))


res2_1 = np.vstack((oof_pref2, sub2))
res2_1 = pd.DataFrame(res2_1)
# res2_1.to_csv("res2.csv", index=False)

res1.index = range(len(res1))
res2_1.index = range(len(res2_1))
final_1 = res2_1.copy()
final_2 = res2_1.copy()
for i in range(11):
    final_1[i] = res1['sex1']*res2_1[i]
    final_2[i] = res1['sex2']*res2_1[i]
id_list = pd.concat([train[['device_id']], test[['device_id']]])
final = id_list
final.index = range(len(final))
final.columns = ['DeviceID']
final_pred = pd.concat([final_1, final_2], 1)
final = pd.concat([final, final_pred], 1)
final.columns = ['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6',
                 '1-7', '1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4',
                 '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']

final.to_csv('/Users/xuelili/eguan data/feature1/yg_best_nn.csv', index=False)



/Users/xuelili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:66: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/xuelili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:89: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


FOLD |  1


TypeError: while_loop() got an unexpected keyword argument 'maximum_iterations'